Example 1: Comparing KRAS Proteins
===================================


In [6]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1' 
import numpy as np

#from GW_prot import *


In [7]:
import sys
sys.path.insert(0,'../src')
from FGW_protein import *

Many of the PDB files in this dataset contain multple chains; we need to specify which chain is KRAS protein.

(This dataset has been curated as follows.....XXXXXX)

In [8]:
file_list = ['4dsn.pdb', '4dso.pdb', '4dst.pdb', '4epr.pdb', '4ept.pdb', '4epv.pdb', '4epw.pdb', '4epx.pdb','4epy.pdb', '4ldj.pdb',
 '4lpk.pdb', '4obe.pdb', '4obe.pdb', '4pzy.pdb', '4pzy.pdb', '4ql3.pdb', '4tq9.pdb', '4tq9.pdb', '4tqa.pdb', '4tqa.pdb', '5mla.pdb',
 '5mlb.pdb', '5mlb.pdb', '5mlb.pdb', '5mlb.pdb', '5o2s.pdb', '5o2s.pdb', '5o2s.pdb', '5o2s.pdb', '5o2t.pdb', '5ocg.pdb', '5tar.pdb',
 '5ufe.pdb']
chain_list = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 
              'A', 'B', 'A', 'B', 'A', 'A', 'C', 'E', 'G', 'A', 'C', 'E', 'G', 'A', 'A', 'A', 'A']

We start by creating FGW_protein objects for each KRAS protein.

In [9]:
protein_list = [FGW_protein.make_protein_from_pdb(
    pdb_file = os.path.join("Examples Data/Example 1 - KRAS/" , file_list[i]), 
    chain_id = chain_list[i])
    for i in range(33)] # should be 33, 5 for debugging

We then initialize a `Stress_Comparison` object to compare these proteins using GW and their stresses.
In this example we store the transport plans in RAM for simplicity. This will use roughly XXXX MB. 

In [10]:
import stress_comparison
A = stress_comparison.Stress_Comparison(prot_list = protein_list, RAM =  True)

We then run GW all-vs-all. On our computer with 16 parallel processes this take XXXX. With a single process it takes XXXXXX. 

In [11]:
#!pip show objsize
import sys
sys.path.append('/home/elijah/pymol/lib/python3.10/site-packages')
import objsize

In [14]:
objsize.get_deep_size(A) #7873301 raw, 130 379 581 after calc

130379581

In [13]:
%%time
A.GW_compute_stresses(processes = 1) #31s for all 33 of them

4dsn_A 4dso_A 0.2427586536280204
4dsn_A 4dst_A 0.10742734071376632
4dsn_A 4epr_A 1.9101268354530687
4dsn_A 4ept_A 1.8916865860495578
4dsn_A 4epv_A 1.6647623268116094
4dsn_A 4epw_A 1.658091131303196
4dsn_A 4epx_A 1.6627068681565702
4dsn_A 4epy_A 1.6621996649151063
4dsn_A 4ldj_A 1.6707748175582684
4dsn_A 4lpk_B 1.953266287898907
4dsn_A 4obe_A 1.7075876788173363
4dsn_A 4obe_B 1.681946929171668
4dsn_A 4pzy_A 1.6707522657859566
4dsn_A 4pzy_B 1.6711644619529828
4dsn_A 4ql3_A 1.668777255592884
4dsn_A 4tq9_A 1.7263850088643913
4dsn_A 4tq9_B 1.7233237107322026
4dsn_A 4tqa_A 1.73972611403086
4dsn_A 4tqa_B 1.7375896652916298
4dsn_A 5mla_A 1.9194096981152644
4dsn_A 5mlb_A 1.918015811754422
4dsn_A 5mlb_C 1.925254165230974
4dsn_A 5mlb_E 1.9199183075648383
4dsn_A 5mlb_G 1.9452059638226054
4dsn_A 5o2s_A 1.918015811754422
4dsn_A 5o2s_C 1.925254165230974
4dsn_A 5o2s_E 1.9199183075648383
4dsn_A 5o2s_G 1.9452059638226054
4dsn_A 5o2t_A 1.9194096981152644
4dsn_A 5ocg_A 1.4057737139533304
4dsn_A 5tar_A 1.931

In [8]:
%%time 

A.GW_compute_stresses(processes = 16)  #~5s

4dsn_A 4dso_A 0.2427586536280204
4dsn_A4dsn_A  5mlb_A4dst_A  1.9180158117544220.10742734071376632

4dso_A 4lpk_B 2.0011195539128446
4dso_A 5ocg_A 
1.43074339035487794dsn_A 5mlb_C 1.925254165230974
4dst_A 5mla_A 1.93822237858595434dsn_A4epr_A
  4obe_A 0.7682522643575901
4epr_A 1.91012683545306874dso_A
 4obe_A 1.7548444231032563
4ept_A 4epr_A 4dso_A4epr_A 5tar_A  5tar_A1.9004213609134866
4epv_A5mlb_E4dst_A    4tq9_A3.244775987260605 4epw_A5mlb_A0.5212692670345069

0.45463096845755274 
4epw_A  4obe_B4dsn_A1.93249056772024
5ocg_A4lpk_B 0.7999971139223911 
  0.81663009557543684epx_A4dso_A5mlb_E
0.849093794728709  1.91991830756483834obe_B 1.725109491411348 4dsn_A5o2s_A
  
4ept_A0.889396173511935 
1.89168658604955784epy_A
4epr_A
  5ufe_A4ept_A  5mlb_A0.47781768340684694dso_A 5ufe_A  4ldj_A
0.88843853648002071.92996172983140654lpk_B

5mlb_G4epv_A  4dst_A 4tq9_A 4tqa_A4epr_A0.6178680347857012
 4epx_A  0.4532051695781388
0.76997636398581045mlb_C4obe_A4pzy_A
   1.93972655586114785o2s_C
 0.8919027

In [9]:
A

We can study how the conformation of KRAS changes depending on the ligand it is bound to. These PDB files contain KRAS bound to guanosine diphosphate (GDP) and those bound to guanosine triphosphate-analogues (GNP, GCP, GSP).

In [49]:
ligand_list = ['GCP', 'GSP', 'GCP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP',  'GDP',
 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GSP', 'GDP', 'GDP',
 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GDP', 'GSP', 'GNP', 'GDP', 'GNP']

For this visual analysis, we will create a UMAP plot, with different colors corresponding to different ligands. 

In [ ]:
from umap.umap_ import UMAP
import matplotlib.pyplot as plt



distance_matrix = A.get_GW_dmat()

reducer = UMAP(metric = 'precomputed', random_state=1)

embedding = reducer.fit_transform(distance_matrix)
    
        
plt.figure(figsize=(8, 8))
ligand_color_dict = {'GDP' : 'red',  'GSP': 'green', 'GNP': 'purple', 'GCP' : 'cyan', 'X': 'black'}
plt.scatter(embedding[:, 0], embedding[:, 1], co= [ligand_color_dict[l] for l in ligand_list], s=5)

plt.show()

        

Next we use pymol to visualize the proteins and the stress levels of different residues in the proteins. 
`A.raw_stress_levels` contains the stresses of all the pairwise calculations, so we first average them for each protein.

In [ ]:
avg_stresses_dict =  stress_comparison.normalize_stress_dict( A.raw_stress_dict)
avg_stresses_list = [avg_stress_dict[name] for name in A.name_list]

pymol_protein_viewer.show_proteins_with_values(
    infiles = ["Examples Data/Example 1 - KRAS/" + file for file in file_list,
    chain_ids = chain_list,
    data_lists = avg_stresses_list,
    outfile = "Examples Data/Example 1 - KRAS/KRAS stresses.pse")

The pymol file we generate can then be openned a the pymol GUI window.

For clarity we can also view a single protein at a time.



Next we will test how well the stress levels can be used to predict the switch I and switch II regions. These regions do not have well-defined endpoints but we will choose to use residues 30-40 and 60-72 in the canonical indexing respectively. 